In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 
from pyspark.sql.window import Window 
spark = SparkSession.builder.getOrCreate()
#data = spark.read.parquet("/Users/vinnymac/Library/CloudStorage/OneDrive-Personal/Dataset/log_search/20220701")
#/Users/vinnymac/Library/CloudStorage/OneDrive-Personal/Dataset/log_search/20220714

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/24 11:46:59 WARN Utils: Your hostname, Vinhs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.177 instead (on interface en0)
26/01/24 11:46:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/24 11:47:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_07  = spark.read.parquet("/Users/vinnymac/Library/CloudStorage/OneDrive-Personal/Dataset/log_search/20220701")

In [3]:
df_07 = spark.read.parquet("/Users/vinnymac/Library/CloudStorage/OneDrive-Personal/Dataset/log_search/20220714")

In [4]:
def process_log_search_data(df):
    df = df.select('user_id','keyword')
    df = df.filter((col('user_id').isNotNull()) & (col('keyword').isNotNull())  )
    df = df.groupBy('user_id','keyword').count().withColumnRenamed('count','frequency')
    w = Window.partitionBy('user_id').orderBy(col('frequency').desc())
    df = df.withColumn("rank",row_number().over(w))
    df = df.filter(col('rank') == 1)
    df = df.select('user_id','keyword').withColumnRenamed('keyword','most_searched')
    df = df.orderBy('user_id',ascending = False )
    return df

In [10]:
output_path = "/Users/vinnymac/Documents/Dev/Apache-Spark/Spark/DE Project/code/Customer Behavior/map/input.csv"
output = ms_07.union(ms_07).select('most_searched').distinct()
output.toPandas().to_csv(output_path, index=False)

In [13]:
### Run AT to get Labelled data

DataFrame[user_id: string, most_searched: string]

In [17]:
mapping = spark.read.csv('/Users/vinnymac/Documents/Dev/Apache-Spark/Spark/DE Project/code/Customer Behavior/map/classified_genres_output.csv',header = True)
mapping.show()

+--------------------+---------+
|               movie|    genre|
+--------------------+---------+
|               anime|Animation|
|chuyển sinh thành...|Animation|
|       OUR BELOVE SS|  Romance|
|                hope|    Drama|
|   nhat da tan nuong|    Drama|
|     tro lai tuoi 18|    Drama|
|aikatsu - nhiệt h...|    Drama|
|      su ma cua zero|   Horror|
|               luffy|Animation|
|    cô nàng ngổ ngáo|    Drama|
|Cô Nàng Đầu Bếp C...|    Drama|
|cuộc sống thượng ...|    Drama|
|    bong da viet nam|   Sports|
|             lý thần|    Drama|
|viet nam và afgan...|    Drama|
|tuyển tập: năm ch...|    Drama|
|  nghệ thuạt săn quy|   Horror|
|             TSURUNE|    Drama|
|     cartoon network|    Drama|
|         mỹ nhân ngư|    Drama|
+--------------------+---------+
only showing top 20 rows


In [24]:
#Map 07
msg_07 = (
    ms_07
    .join(broadcast(mapping), col("most_searched") == col("movie"), how="inner")
    #.withColumn("genre", coalesce(col("genre"), lit("Other")))
    .drop("movie")
    .withColumnRenamed("most_searched","most_searched_07")
    .withColumnRenamed("genre","genre_07")
    
)
msg_07.show()

+-------+--------------------+----------+
|user_id|    most_searched_06|  genre_06|
+-------+--------------------+----------+
|0005063|                vtv5|TV Channel|
|0005117|           YOUTUBIII|     Drama|
|0005178|   bored to death s1|     Drama|
|0006773|                  30|     Drama|
|0007729|              KUROKO|     Drama|
|0007996|   phàm nhân tu tiên|     Drama|
|0010414| bật thầy hoá trang |     Drama|
|0014661|        gã không mặt|     Drama|
|0016455|                 cây|     Drama|
|0017222|            BAO NGAM|     Drama|
|0018591|Nhanh Và Nguy Hiểm 9|     Drama|
|0023968|          bé học chữ|     Drama|
|0027338|          fairy tail|     Drama|
|0028131|                  on|     Drama|
|0028736|    kimetsu no yaiba|     Drama|
|0030912|    trái tim bé bỏng|     Drama|
|0031332|trò chơi vương qu...|    Horror|
|0031911|     the sixth sense|     Drama|
|0033766| thương ngày nắng về|     Drama|
|0033788|những chàng trai ...|   K Drama|
+-------+--------------------+----

In [25]:
#Map 07
msg_07 = (
    ms_07
    .join(broadcast(mapping), col("most_searched") == col("movie"), how="inner")
    #.withColumn("genre", coalesce(col("genre"), lit("Other")))
    .drop("movie")
    .withColumnRenamed("most_searched","most_searched_07")
    .withColumnRenamed("genre","genre_07")
    
)
msg_07.show()

+-------+--------------------+---------+
|user_id|    most_searched_07| genre_07|
+-------+--------------------+---------+
|0001893|    Kẻ Cắp Mặt Trăng|    Drama|
|0003361|         lộc đỉnh ký|    Drama|
|0004007|        bật quân đội|    Drama|
|0004285|                 eve|    Drama|
| 000496|  Gia đình Điệp Viên|    Drama|
|0008802|      người ấy là ai|    Drama|
|0009775|             kingdom|    Drama|
|0011135|nhạc sinh nhật ka...|    Music|
|0012770|         kênh svts24|    Drama|
|0012798|           siêu nhân|    Drama|
|0012800| thôn phệ tinh không|     News|
|0015713|           penthouse|    Drama|
|0018156|         hội pháp sư|    Drama|
|0021617|  xích cước đại tiên|    Drama|
|0023970|khúc biến tấu ánh...|    Drama|
|0025145|            vincenzo|    Drama|
|0027873|               moana|    Drama|
|0029834|               ANIME|Animation|
|0030030|           KAKEGURUI|    Drama|
|0030676|   săn lùng thỏ ngọc|    Drama|
+-------+--------------------+---------+
only showing top

In [29]:
trend = (msg_06.join(msg_07, on = "user_id",how = "inner")
                 .withColumn("Trending Type",when(msg_06.genre_06 == msg_07.genre_07,"Unchanged").otherwise("Changed"))
                 .withColumn("Previous", when(col("Trending Type")=="Unchanged","Unchanged").otherwise(col("most_searched_06")))
    
)
trend.show()

+--------+--------------------+---------+--------------------+--------+-------------+--------------------+
| user_id|    most_searched_06| genre_06|    most_searched_07|genre_07|Trending Type|            Previous|
+--------+--------------------+---------+--------------------+--------+-------------+--------------------+
| 0250871|cái bống sư tử mè...|    Drama|     điên thì có sao|   Drama|    Unchanged|           Unchanged|
| 0260217|    yêu nhầm chị dâu|  Romance|kẻ trộm mặt trăng...|   Drama|      Changed|    yêu nhầm chị dâu|
| 0392220|        mộng hoa lục|  C Drama|          nữ luật sự|   Drama|      Changed|        mộng hoa lục|
| 0589053|             NHAT DA|    Drama|              DAI DU|   Drama|    Unchanged|           Unchanged|
|06400675|      LOVE DEATH AND|  Romance|              VAN TI|   Drama|      Changed|      LOVE DEATH AND|
|06402961|              NARUTO|Animation| chú thuật hồi chiến|   Drama|      Changed|              NARUTO|
|06408906|em đẹp nhất là kh...|    Dr

26/01/25 01:18:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1061416 ms exceeds timeout 120000 ms
26/01/25 01:18:26 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/25 01:34:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$